## 一. 读到partial message
&nbsp;&nbsp;&nbsp;&nbsp;当从SelectableChannel中读取一块数据时,并不知道这块数据是比1条message的数据多还是少. 
一个数据块可能包含3种数据: 部分信息(短于1条message),正好一条message, 多于1条message; 处理partial message要注意两点:
<img src="img/non-blocking-server-6.png" width="48%" height="48%" align="right">
     1. 如何发现数据块中是否包含一条完整的message
     2. 在其他数据到来之前,如何处理这些partial message

&nbsp;&nbsp;&nbsp;&nbsp;对于第一点, 只要发现数据块中的数据大于或等于一条完整message, 这个message就会被发送到pipeline中处理. 检查是否包含完整message的动作要反复执行; 对于第二点, 读到partial message时要将他保存, 直到其余信息到达channel中.    
  
&nbsp;&nbsp;&nbsp;&nbsp;检查full messages和保存partial messages都是Message Reader的职责, 为了避免从不同channel中读取的消息混在一起, 对每一个channel使用一个Message Reader. 这种设计看起来如右图:  
  
当然, Message Reader需要满某种特定协议, 协议用来描述将要读取到的message的格式.如果想要Message Reader支持多种消息格式, 则需要Message Reader实现可配置协议的功能.

## 二. 保存partial message
### 1. Resize by Copy
一旦一个message被pipeline完全处理完毕, 就要去释放其buffer占用的内存, 下次从这个连接中收到数据时, 再次从最小的buffer size开始

### 3. TLV加密信息
&nbsp;&nbsp;&nbsp;&nbsp;TLV格式(type,length,value).一些协议支持把message封装成TLV格式, 这意味着, message的头部存储着mesage的长度, 接收端立刻就能知道为该mesage分配多大的内存来存储  
&nbsp;&nbsp;&nbsp;&nbsp;TLV格式让内存管理变得很容易, 且没有一点内存浪费.   
&nbsp;&nbsp;&nbsp;&nbsp;TLV格式的一个缺点是, 在消息全部抵达接收端以前, 就为该message分配全部内存; 如果一个速度很慢的连接发送一个长度很大的message', 会让接受者立刻分配其剩余的所有内存, 导致服务无响应


## 三. Writing Partial Messages

### 四. RingBuffer
#### 1. wrapping around  
&nbsp;&nbsp;&nbsp;&nbsp;Ring Buffer可以被用作queue,本质上是一个固定长度的数组. 包含一个read position和一个write position, 分别表示下次从该位置读数据和下次再该位置写数据. 当write position到达数组的末尾时,置回为0, read position也一样, 这种position到达数组末尾就置回0的操作称作"wrapping around", 该操作把一个array变成RingBuffer

#### 2. free space和used space  
1. 当write position没有发生"wrapping around"时    
     used space="write position"和"read position"之间的区域    
     剩下的为used space

2. 当write position发生"wrapping around"时   
     free space="write position"和"read position"之间的区域  
     剩下的微free space
<img src="img/ringbuffer11.png" width="50%" height="50%">

#### 3. RingBuffer实现  
ringbuffer有很多实现方式, 其中最简单的两种: 
- Using a fill count
- Using a flip marker

#### 4. fill count
&nbsp;&nbsp;&nbsp;&nbsp;fill count方式使用3个标记: capacity, write position和fill count; fillcount表示ring buffer中存在多少可用元素.  
&nbsp;&nbsp;&nbsp;&nbsp;插入数据时, 先查看fill count是否已满, 如果未满则插入, 并把write position向后移动  
&nbsp;&nbsp;&nbsp;&nbsp;读取数据时, 先查看fill count是否为0, 如果不为0, 则从`write position减fill count`的位置开始读取数据  
&nbsp;&nbsp;&nbsp;&nbsp;如下使用available变量表示fill count

#### 5. Flip Marker